# First Level Analysis

I start the investigation by conducting first-level analysis to obtain BOLD parameter estimates from individuals. There are many fMRI tools that can achieve this, and many of them have convenient graphical user interfaces (GUIs). Here, for reproducibility, we follow a similar pipeline as the original paper by using a combination of `fmriprep` and `nipype` with FSL and AFNI integrations. Both `fmriprep` and `nipype` are powerful, and the latter is highly flexible in terms of customizability compared to its GUI counterparts (such as FSL and AFNI).

Nevertheless, the system environment is different, and we won't go into details on the setup process but will only provide a few caveats I encountered during setup:

1. Using a Docker container like `neurodocker` is more reproducible and avoids many installation and version management problems. I used a non-containerized version just for the ease of coding and brain image inspections (provided by FSLeyes).

2. The FSL and AFNI packages I installed on my Arch Linux distribution do not provide any PATH integration, so I needed to manually add the FSL executable folder to my `PATH` variable.

3. I also installed the SPM standalone package but did not use SPM in any of the subsequent analysis. The SPM standalone requires extra setup code, which is provided below for anyone and my future self as a reference.

In [2]:
# In case we need to run SPM, here's the setup
# %load setup_spm.py
# from nipype.interfaces import spm

# MATLAB_RUNTIME = '/home/rongfei/MATLAB/runtime/R2024b'
# SPM_START_SCRIPT = '/home/rongfei/Builds/spm12/spm_standalone/run_spm25.sh'

# MCR_COMMAND = f'{SPM_START_SCRIPT} {MATLAB_RUNTIME} script'
# print("MCR_COMMAND: ", MCR_COMMAND)

# spm.SPMCommand.set_mlab_paths(MCR_COMMAND, use_mcr=True)
# print(spm.SPMCommand().version)

In [3]:
from nipype.interfaces import fsl

# check FSL integration

print("Check if FSL is runnable... ", end='')
print('ok' if fsl.check_fsl() == 0 else 'not ok')

Check if FSL is runnable... ok


The following code block creates a customized `SubjectInfo` class that provides necessary brain image and event-related file information for later processing. Here I briefly list some of the class methods and their corresponding purposes:

1. `__init__` constructor: Creates parent folder reference
2. `filter_confound(list)`: Provides confounds dataframe produced by fMRIPrep
3. `create_fsl_explanatory_variable_file`: Loads events from the event data file and converts into FSL 3-column format
4. `create_fsl_evs_scap`: Separates different event conditions and applies the method above to create explanatory variables (EVs)

Alternatively, we can use the `DataGrabber` and `IdentityInterface` classes provided by `nipype` to achieve the same functionalities and maintain design style consistency. However, I simply prefer writing my own class for more customization and better readability.

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
import os

# Create a class to handle subject information
class SubjectInfo:
    def __init__(
        self,
        subject_id: str,
        raw_data_dir: Path | None = None,
        derivatives_dir: Path | None = None,
        working_dir: Path | None = None,
        results_dir: Path | None = None,
    ):
        if raw_data_dir is None:
            raw_data_dir = Path("data")
        if derivatives_dir is None:
            derivatives_dir = Path("derivatives")
        if working_dir is None:
            working_dir = Path("working")
        if results_dir is None:
            results_dir = Path("results")

        working_dir.mkdir(parents=True, exist_ok=True)

        self.subject_id: str = subject_id
        self.raw_data_dir: Path = raw_data_dir
        self.derivatives_dir: Path = derivatives_dir
        self.working_dir: Path = working_dir
        self.results_dir: Path = results_dir

    def get_confounds_file(self, task_name: str):
        confounds_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_confounds.tsv"
        )
        if not confounds_file.exists():
            raise FileNotFoundError(f"Confounds file not found: {confounds_file}")
        return confounds_file

    def get_event_file(self, task_name: str):
        event_file = (
            self.raw_data_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_events.tsv"
        )
        if not event_file.exists():
            raise FileNotFoundError(f"Event file not found: {event_file}")
        return event_file

    def get_subject_id(self):
        return self.subject_id

    def filter_confounds(self, task_name: str, confounds_columns: list[str]):
        confounds_file = self.get_confounds_file(task_name)
        confounds_df = pd.read_csv(confounds_file, sep="\t", na_values="n/a")
        selected_columns = confounds_df[confounds_columns]

        # map na to 0
        selected_columns = selected_columns.fillna(0).astype(float)

        selected_columns = selected_columns.round(3)

        # get the file name without the extension and add filtered to the end before the extension
        confounds_folder = self.working_dir / self.subject_id
        confounds_folder.mkdir(parents=True, exist_ok=True)

        confounds_file_name = (
            confounds_folder
            / f"{self.subject_id}_task-{task_name}_confounds_filtered.tsv"
        )
        selected_columns.to_csv(confounds_file_name, sep=" ", header=False, index=False)
        return confounds_file_name
    
    def get_anatomical_file(self):
        anat_file = (
            self.derivatives_dir
            / self.subject_id
            / "anat"
            / f"{self.subject_id}_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz"
        )
        if not anat_file.exists():
            raise FileNotFoundError(f"Anatomical file not found: {anat_file}")
        return anat_file

    def get_standard_brain_mask(self, task_name: str):
        mask_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz"
        )
        if not mask_file.exists():
            raise FileNotFoundError(f"Mask file not found: {mask_file}")
        return mask_file

    def get_transformed_brain_mask(self, task_name: str):
        mask_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_brainmask_processed.nii.gz"
        )
        return mask_file

    def get_processed_bold_file(self, task_name: str):
        bold_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz"
        )
        if not bold_file.exists():
            raise FileNotFoundError(f"Bold file not found: {bold_file}")
        return bold_file

    def get_smoothed_bold_file(self, task_name: str):
        bold_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_preproc_smoothed.nii.gz"
        )
        return bold_file

    def get_events(self, task_name: str):
        events_file = self.get_event_file(task_name)
        if not events_file.exists():
            raise FileNotFoundError(f"Events file not found: {events_file}")

        events_df = pd.read_csv(events_file, sep="\t", na_values="n/a")
        return events_df
    
    def get_working_task_dir(self, task_name: str):
        task_dir = self.working_dir / self.subject_id / task_name
        task_dir.mkdir(parents=True, exist_ok=True)
        return task_dir
    
    def get_output_dir(self):
        task_dir = self.results_dir / self.subject_id 
        task_dir.mkdir(parents=True, exist_ok=True)
        return task_dir

    def create_fsl_explanatory_variable_file(
        self,
        task_name: str,
        events_df: pd.DataFrame,
        ev_name: str,
        reference_duration_column: str | None = None,
        reference_amplitude_column: str | None = None,
        fixed_duration: float = 1.0,
        fixed_amplitude: float = 1.0,
    ):
        events_df = events_df.dropna(subset=["onset"])
        if events_df.empty:
            warnings.warn(f"No events found in the events file for {ev_name}")
            


        onsets = events_df["onset"].to_list()
        if reference_duration_column is not None: 
            # if we have a reference duration column
            durations = events_df[reference_duration_column].to_list()
        else: 
            # if we don't have a reference duration column, use a fixed duration
            durations = [fixed_duration] * len(onsets)

        if reference_amplitude_column is not None:
            # center the amplitude around 0
            amplitudes = events_df[reference_amplitude_column]
            amplitudes = (
                amplitudes.apply(lambda x: x - np.mean(amplitudes)).round(3).to_list()
            )

        else:
            amplitudes = [fixed_amplitude] * len(onsets)

        fsl_explanatory_variable_df = pd.DataFrame(
            {"0": onsets, "1": durations, "2": amplitudes}
        )

        ev_file_folder = self.working_dir / self.subject_id / task_name 
        ev_file_folder.mkdir(parents=True, exist_ok=True)
        ev_file_path = ev_file_folder / f"{ev_name}.txt"

        fsl_explanatory_variable_df.to_csv(
            ev_file_path,
            sep="\t",
            header=False,
            index=False,
        )

        return ev_file_path


    def create_fsl_evs_scap(self):
        events_df = self.get_events("scap")

        # separate correct and incorrect events
        correct_events = events_df[events_df["ResponseAccuracy"] == "CORRECT"]
        incorrect_events = events_df[events_df["ResponseAccuracy"] == "INCORRECT"]

        evs = []
        n_regressors = 25 # number of regressors as stated in the original paper

        # initialize orthogonalization dictionary to remove reaction time correlation PS: a weird FSL format
        orthogonalization = {x: {y:0 for y in range(1,n_regressors+1)} for x in range(1,n_regressors+1)}

        # load and delay levels as stated in the original paper
        load_levels = [1, 3, 5, 7]
        delay_levels = [1.5, 3.0, 4.5]

        for load_level in load_levels:
            for delay_level in delay_levels:
                condition_mask = (correct_events["Load"] == load_level) & (
                    correct_events["Delay"] == delay_level
                )
                condition_events = correct_events[condition_mask]

                if condition_events.empty:
                    warnings.warn(
                        f"No events found for load {load_level} and delay {delay_level}"
                    )
                
                # condition_events["onset"] = condition_events["onset"] + delay_level
                condition_events.loc[:, "onset"] = condition_events["onset"] + delay_level

                fixed_ev = self.create_fsl_explanatory_variable_file(
                    "scap",
                    condition_events,
                    f"LOAD{load_level}_DELAY{delay_level}",
                    fixed_duration=3.0,
                    fixed_amplitude=1.0,
                )

                reaction_time_ev = self.create_fsl_explanatory_variable_file(
                    "scap",
                    condition_events,
                    f"LOAD{load_level}_DELAY{delay_level}_rt",
                    reference_duration_column="ReactionTime",
                    fixed_amplitude=1.0,
                )

                evs.append(fixed_ev)
                fix_index = len(evs) 

                evs.append(reaction_time_ev)
                reaction_time_index = len(evs)
                orthogonalization[reaction_time_index][fix_index] = 1
                orthogonalization[reaction_time_index][0] = 1


        
        if not incorrect_events.empty:
            error_ev = self.create_fsl_explanatory_variable_file(
                "scap",
                incorrect_events,
                "Error",
                fixed_duration=3.0,
                fixed_amplitude=1.0,
            )
            evs.append(error_ev)
        

        valid_ev_files = []
        for ev_file in evs:
            if os.path.exists(ev_file) and os.path.getsize(ev_file) > 0:
                valid_ev_files.append(ev_file.absolute().as_posix())
            else:
                print(f"Warning: Empty or missing EV file: {ev_file}")
        
        # Validation
        expected_regressors = len(load_levels) * len(delay_levels) * 2 + 1  # 25
        if len(valid_ev_files) != expected_regressors:
            print(f"Warning: Expected {expected_regressors} regressors, got {len(valid_ev_files)}")

        
        return {
            'event_files': valid_ev_files,
            'orthogonalization': orthogonalization
        }


Next, I create the 20+20 (bidirectional) contrasts stated by the original paper. 

In [5]:
def create_scap_contrast():
    contrasts = []

    # All conditions
    contrasts += [
        ("All","T",["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5","LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5","LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5","LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5"],
            [1] * 12,
        )
    ]

    contrasts += [
        ("All_rt","T",["LOAD1_DELAY1.5_rt","LOAD1_DELAY3.0_rt","LOAD1_DELAY4.5_rt","LOAD3_DELAY1.5_rt","LOAD3_DELAY3.0_rt","LOAD3_DELAY4.5_rt","LOAD5_DELAY1.5_rt","LOAD5_DELAY3.0_rt","LOAD5_DELAY4.5_rt","LOAD7_DELAY1.5_rt","LOAD7_DELAY3.0_rt","LOAD7_DELAY4.5_rt"],
            [1] * 12,
        )
    ]

    # # Load effects
    contrasts += [
        ("Load1","T", ["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5"], [1] * 3)
    ]
    contrasts += [
        ("Load3","T", ["LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5"], [1] * 3)
    ]
    contrasts += [
        ("Load5","T", ["LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5"], [1] * 3)
    ]
    contrasts += [
        ("Load7","T", ["LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5"], [1] * 3)
    ]

    # # Delay effects
    contrasts += [
        ("Delay1.5","T",["LOAD1_DELAY1.5","LOAD3_DELAY1.5","LOAD5_DELAY1.5","LOAD7_DELAY1.5"],
            [1] * 4,
        )
    ]
    contrasts += [
        ("Delay3","T",["LOAD1_DELAY3.0","LOAD3_DELAY3.0","LOAD5_DELAY3.0","LOAD7_DELAY3.0"],
            [1] * 4,
        )
    ]
    contrasts += [
        ("Delay4.5","T",["LOAD1_DELAY4.5","LOAD3_DELAY4.5","LOAD5_DELAY4.5","LOAD7_DELAY4.5"],
            [1] * 4,
        )
    ]

    # Parametric contrasts
    contrasts += [
        ("LinearUp_load","T",["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5","LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5","LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5","LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5",            ],
            [-3] * 3 + [-1] * 3 + [1] * 3 + [3] * 3,
        )
    ]

    contrasts += [
        ("LinearUp_delay","T",["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5","LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5","LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5","LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5",            ],
            [-1, 0, 1] * 4,
        )
    ]

    # Pairwise load comparisons
    contrasts += [
        ("Load3-load1","T",["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5","LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5",            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Load5-load1","T",["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5","LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5",            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Load7-load1","T",["LOAD1_DELAY1.5","LOAD1_DELAY3.0","LOAD1_DELAY4.5","LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5",            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Load5-load3","T",["LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5","LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5",],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Load7-load3","T",["LOAD3_DELAY1.5","LOAD3_DELAY3.0","LOAD3_DELAY4.5","LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5",],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Load7-load5","T",["LOAD5_DELAY1.5","LOAD5_DELAY3.0","LOAD5_DELAY4.5","LOAD7_DELAY1.5","LOAD7_DELAY3.0","LOAD7_DELAY4.5",],
            [-1, -1, -1, 1, 1, 1],
        )
    ]

    # Pairwise delay comparisons
    contrasts += [
        ("Delay4_5-delay1_5","T",["LOAD1_DELAY1.5","LOAD3_DELAY1.5","LOAD5_DELAY1.5","LOAD7_DELAY1.5","LOAD1_DELAY4.5","LOAD3_DELAY4.5","LOAD5_DELAY4.5","LOAD7_DELAY4.5",            ],
            [-1, -1, -1, -1, 1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Delay3-delay1_5","T",["LOAD1_DELAY1.5","LOAD3_DELAY1.5","LOAD5_DELAY1.5","LOAD7_DELAY1.5","LOAD1_DELAY3.0","LOAD3_DELAY3.0","LOAD5_DELAY3.0","LOAD7_DELAY3.0",            ],
            [-1, -1, -1, -1, 1, 1, 1, 1],
        )
    ]
    contrasts += [
        ("Delay4_5-delay3","T",["LOAD1_DELAY3.0","LOAD3_DELAY3.0","LOAD5_DELAY3.0","LOAD7_DELAY3.0","LOAD1_DELAY4.5","LOAD3_DELAY4.5","LOAD5_DELAY4.5","LOAD7_DELAY4.5",            ],
            [-1, -1, -1, -1, 1, 1, 1, 1],
        )
    ]

    # Create bidirectional contrasts (positive and negative)
    bidirectional_contrasts = []
    for name, contrast_type, conditions, weights in contrasts:
        # Add original
        bidirectional_contrasts.append((name, contrast_type, conditions, weights))

        if"-" in name:
            # Reverse:"A-B" becomes"B-A"
            parts = name.split("-", 1)
            neg_name = f"{parts[1]}-{parts[0]}"
        else:
            # Add neg_ prefix
            neg_name = f"neg_{name}"

        bidirectional_contrasts.append(
            (neg_name, contrast_type, conditions, [-w for w in weights])
        )

    contrasts = bidirectional_contrasts
    return contrasts

In [6]:
contrasts = create_scap_contrast()
for i, contrast in enumerate(contrasts):
    print(f"{i+1}: {contrast[0]}")






1: All
2: neg_All
3: All_rt
4: neg_All_rt
5: Load1
6: neg_Load1
7: Load3
8: neg_Load3
9: Load5
10: neg_Load5
11: Load7
12: neg_Load7
13: Delay1.5
14: neg_Delay1.5
15: Delay3
16: neg_Delay3
17: Delay4.5
18: neg_Delay4.5
19: LinearUp_load
20: neg_LinearUp_load
21: LinearUp_delay
22: neg_LinearUp_delay
23: Load3-load1
24: load1-Load3
25: Load5-load1
26: load1-Load5
27: Load7-load1
28: load1-Load7
29: Load5-load3
30: load3-Load5
31: Load7-load3
32: load3-Load7
33: Load7-load5
34: load5-Load7
35: Delay4_5-delay1_5
36: delay1_5-Delay4_5
37: Delay3-delay1_5
38: delay1_5-Delay3
39: Delay4_5-delay3
40: delay3-Delay4_5


Now comes the actual workflow

![workflow](figs/first_level.png)

- The workflow applies the brain mask to specify the valid brain region, then applies AFNI 3D spatial smoothing to enhance the signal.
- Next, the selected confounds (motion parameters) from fMRIPrep are passed to the model as `realignment_parameter` for additional artifact removal. The orthogonalization dictionary (to remove covariance from reaction time regressors) is also passed for accurate modeling.
- The model uses the double gamma derivative HRF as the paper suggests.

In [5]:
# Create a workflow function

import nipype.pipeline.engine as pe
import nipype.interfaces.fsl as fsl
import nipype.interfaces.afni as afni
from nipype.algorithms.modelgen import SpecifyModel
import shutil


def subject_level_modeling(subject_info: SubjectInfo, task_name='scap'):
    workflow = pe.Workflow(name="first_level")
    
    # Define the input nodes to accept inputs of the following:
    # - functional image
    # - brain mask
    # - confounds file


    in_file = subject_info.get_processed_bold_file(task_name).absolute().as_posix()
    mask_file = subject_info.get_standard_brain_mask(task_name).absolute().as_posix()
    out_file = subject_info.get_processed_bold_file(task_name).absolute().as_posix()

    print(in_file)
    print(mask_file)
    print(out_file)
    apply_mask = pe.Node(
        fsl.maths.ApplyMask(
            in_file=in_file,
            mask_file=mask_file,
            out_file=out_file,
        ),
        name="apply_mask",
    )

    mask_file = subject_info.get_standard_brain_mask(task_name).absolute().as_posix()
    out_file = subject_info.get_smoothed_bold_file(task_name).absolute().as_posix()

    apply_smooth = pe.Node(
        afni.BlurInMask(
            mask=mask_file,
            out_file=out_file,
            fwhm=5.0, # Suggested by original paper
        ),
        name="apply_smooth",
    )

    # Get confounds

    confounds_file = subject_info.filter_confounds(task_name, ['stdDVARS', 'non-stdDVARS', 'vx-wisestdDVARS',
                                 'FramewiseDisplacement', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ'])
    confounds_file = confounds_file.absolute().as_posix()

    # Get regressors (as event files) and orthogonality matrix
    event_files = subject_info.create_fsl_evs_scap()['event_files']
    orthogonalization = subject_info.create_fsl_evs_scap()['orthogonalization']

    model_specification = pe.Node(
        SpecifyModel(
            event_files=event_files,
            realignment_parameters=confounds_file,
            input_units='secs',
            time_repetition=2.0,
            high_pass_filter_cutoff=100,
        ),
        name="model_specification",
    )

    l1design = pe.Node(
        fsl.Level1Design(
            contrasts=create_scap_contrast(),
            orthogonalization=orthogonalization,
            interscan_interval=2.0,
            bases={'dgamma': {'derivs': True}},
            model_serial_correlations=True,
        ),
        name="l1design",
    )

    l1feat = pe.Node(
        fsl.FEATModel(),
        name="l1feat",
    )

    l1estimate = pe.Node(
        fsl.FEAT(),
        name="l1estimate",
    )

    workflow.base_dir = str(subject_info.get_working_task_dir(task_name))
    workflow.connect([
        (apply_mask, apply_smooth, [('out_file', 'in_file')]),
        (apply_smooth, model_specification, [('out_file', 'functional_runs')]),
        (model_specification, l1design, [('session_info', 'session_info')]),
        (l1design, l1feat, [('fsf_files', 'fsf_file'), ('ev_files', 'ev_files')]),
        (l1design, l1estimate, [('fsf_files', 'fsf_file')]), # Note the plural difference
    ])

    # workflow.write_graph(graph2use='colored', format='png', simple_form=True)
    return workflow


In [10]:
def run_workflow(workflow, subject_info, task_name, remove_previous=False):
    workflow.run('MultiProc')

    working_feat_dir = subject_info.get_working_task_dir(task_name) / 'first_level' / 'l1estimate'/ 'run0.feat'

    results_feat_dir = subject_info.get_output_dir() / 'scap.feat'

    results_feat_dir.mkdir(parents=True, exist_ok=True)

    if remove_previous:
        if results_feat_dir.exists():
            shutil.rmtree(results_feat_dir)

    # move working_feat_dir to results_feat_dir
    shutil.move(working_feat_dir, results_feat_dir)

The following block loads the sampled subjects' ids

In [7]:
import tqdm

# read control_subjects_ids.txt and adhd_subjects_ids.txt into lists
control_subjects_ids = pd.read_csv('control_subjects_ids.txt', header=None).iloc[:, 0].tolist()
adhd_subjects_ids = pd.read_csv('adhd_subjects_ids.txt', header=None).iloc[:, 0].tolist()

assert len(control_subjects_ids) == len(adhd_subjects_ids)

ids = []
for i in range(len(control_subjects_ids)):
    ids.append(control_subjects_ids[i])
    ids.append(adhd_subjects_ids[i])





In [14]:
# Notice some ev are missings for some subjects because of the randomization during the experiment. We will remove the subjects in the group level analysis.
# subjects = [SubjectInfo(id) for id in ids]

overwrite = True

for subject_info in tqdm.tqdm(selected_subjects):
    feat_report = subject_info.get_output_dir() / 'scap.feat' / 'report.html'
    if not feat_report.exists():
        workflow = subject_level_modeling(subject_info)
        run_workflow(workflow, subject_info, 'scap')
    elif overwrite:
        shutil.rmtree(subject_info.get_working_task_dir("scap") )
        shutil.rmtree(subject_info.get_output_dir() / 'scap.feat')
        workflow = subject_level_modeling(subject_info)
        run_workflow(workflow, subject_info, 'scap')
    with open("completed_subjects.txt", 'a') as f:
        f.write(f"{subject_info.subject_id}\n")

# subjects = [SubjectInfo("sub-11106")]
# workflow = subject_level_modeling(subjects[0])
# run_workflow(workflow, subjects[0], 'scap', remove_previous=True)

  0%|          | 0/4 [00:00<?, ?it/s]

/home/rongfei/WorkSpace/consortium/derivatives/sub-11106/func/sub-11106_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-11106/func/sub-11106_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-11106/func/sub-11106_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250921-14:48:56,298 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250921-14:48:56,302 nipype.workflow INFO:
	 Running in parallel.
250921-14:48:56,303 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250921-14:48:56,455 nipype.workflow INFO:
	 [Node] Outdated cache found for "first_level.apply_mask".
250921-14:48:56,455 nipype.workflow INFO:
	 [Node] Outdated cache found for "first_level.apply_mask".
250921-14:48:56,579 nipype.workflow INFO:
	 [Node] Settin

 25%|██▌       | 1/4 [02:58<08:54, 178.12s/it]

/home/rongfei/WorkSpace/consortium/derivatives/sub-11044/func/sub-11044_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-11044/func/sub-11044_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-11044/func/sub-11044_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250921-14:51:54,462 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250921-14:51:54,464 nipype.workflow INFO:
	 Running in parallel.
250921-14:51:54,465 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250921-14:51:54,762 nipype.workflow INFO:
	 [Node] Setting-up "first_level.apply_mask" in "/home/rongfei/WorkSpace/consortium/working/sub-11044/scap/first_level/apply_mask".
250921-14:51:54,765 nipype.workflow INFO:
	 [Node] Executing "apply_mask" <nipype.inter

 50%|█████     | 2/4 [05:56<05:56, 178.15s/it]

/home/rongfei/WorkSpace/consortium/derivatives/sub-70001/func/sub-70001_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-70001/func/sub-70001_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-70001/func/sub-70001_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250921-14:54:52,621 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250921-14:54:52,624 nipype.workflow INFO:
	 Running in parallel.
250921-14:54:52,624 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250921-14:54:52,903 nipype.workflow INFO:
	 [Node] Setting-up "first_level.apply_mask" in "/home/rongfei/WorkSpace/consortium/working/sub-70001/scap/first_level/apply_mask".
250921-14:54:52,906 nipype.workflow INFO:
	 [Node] Executing "apply_mask" <nipype.inter

 75%|███████▌  | 3/4 [08:52<02:57, 177.24s/it]

/home/rongfei/WorkSpace/consortium/derivatives/sub-70004/func/sub-70004_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-70004/func/sub-70004_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-70004/func/sub-70004_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250921-14:57:48,783 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250921-14:57:48,786 nipype.workflow INFO:
	 Running in parallel.
250921-14:57:48,787 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250921-14:57:49,70 nipype.workflow INFO:
	 [Node] Setting-up "first_level.apply_mask" in "/home/rongfei/WorkSpace/consortium/working/sub-70004/scap/first_level/apply_mask".
250921-14:57:49,73 nipype.workflow INFO:
	 [Node] Executing "apply_mask" <nipype.interfa

100%|██████████| 4/4 [11:50<00:00, 177.65s/it]


# Analysis Check

The whole analysis would take 

We now check if the statistics image generated from the pipeline matches the original file. Since everything in the pipeline is determinstic, there shouldn't be any major difference (except difference caused by float point precision, if any)

Judge by the output, we've obtained the same results as the original paper.

In [8]:
from matplotlib import pyplot as plt
from nilearn.plotting import plot_stat_map

problematic_subjects = ["sub-10998", "sub-10159", "sub-70055", "sub-70022", "sub-70048", "sub-10680"]

control_subjects_ids = [id for id in control_subjects_ids if id not in problematic_subjects]
adhd_subjects_ids = [id for id in adhd_subjects_ids if id not in problematic_subjects]

# Pick 2 Control and 2 ADHD subjects from the completed subjects list

control_subjects = [SubjectInfo(id) for id in control_subjects_ids[:2]]
adhd_subjects = [SubjectInfo(id) for id in adhd_subjects_ids[:2]]

selected_subjects = control_subjects + adhd_subjects


In [ ]:

# Get the parametic modulation contrast
def get_contrast(subject: SubjectInfo, contrast_name: str):
    anat = subject.get_anatomical_file()
    stat = subject.get_output_dir() / 'scap.feat' /'stats'/ f'{contrast_name}.nii.gz'
    return anat, stat

def get_contrast_downloaded(subject: SubjectInfo, contrast_name: str):
    anat = subject.get_anatomical_file()
    stat =  Path("derivatives/task/") / subject.subject_id / 'scap.feat' /'stats'/ f'{contrast_name}.nii.gz'
    return anat, stat

for subject in selected_subjects:
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
    ax[0].set_title(f'Subject {subject.subject_id} (Ours)')
    anat, cope = get_contrast(subject, 'tstat1')
    plot_stat_map(cope, anat, threshold=3.2, display_mode='z', cut_coords=(-5, 0, 5, 10, 15), colorbar=True, axes=ax[0])
    ax[1].set_title(f'Subject {subject.subject_id} (Original)')
    anat, cope = get_contrast_downloaded(subject, 'tstat1')
    plot_stat_map(cope, anat, threshold=3.2, display_mode='z', cut_coords=(-5, 0, 5, 10, 15), colorbar=True, axes=ax[1])
    plt.tight_layout()
    plt.show()


